In [3]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP/project

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/NLP/project


In [4]:
%pip install -r requirements.txt

     |████████████████████████████████| 8.6 MB 8.7 MB/s 
     |████████████████████████████████| 3.1 MB 25.8 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 61 kB 449 kB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 
     |████████████████████████████████| 395 kB 50.3 MB/s 
     |████████████████████████████████| 428 kB 55.0 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 129 kB 65.3 MB/s 
     |████████████████████████████████| 69 kB 7.2 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jsonschema
    Found 

In [5]:
!nvidia-smi

Wed Dec  1 19:27:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[autoreload of jupyter_client failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'ZMQSocketChannel' from 'jupyter_client.channels' (/usr/local/lib/python3.7/dist-packages/jupyter_client/channels.py)
]
[autoreload of jupyter_client.manager failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: _launch_kernel() requires a code object with 0 free vars, not 1
]
[autoreload of jupyter_client.blocking.client failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: wait_for_ready() requires a code object with 0 free vars, not 1
]
[autoreload

In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn import functional as F
from collections import Counter, namedtuple

import pandas as pd
import seaborn as sns
import numpy as np
import random
import math

import matplotlib.pyplot as plt

from transformers import GPT2LMHeadModel,  GPT2TokenizerFast, GPT2Config, DataCollatorForLanguageModeling, LineByLineTextDataset
from transformers import AdamW, top_k_top_p_filtering
from datasets import load_metric

from tqdm import tqdm
import nltk
from nltk.util import ngrams
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from bert_score import score as BERTscore
from rouge import Rouge

import gpt2
from dataset import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training

## Elon Musk

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/elonmusk \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/elonmusk/train.txt \
  --eval_data_file=data/elonmusk/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

Streaming output truncated to the last 5000 lines.
Memory usage on this node: 7.0/25.5 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/4 CPUs, 1.0/1 GPUs, 0.0/15.02 GiB heap, 0.0/7.51 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2021-11-19_21-22-15
Number of trials: 10/10 (7 PENDING, 1 RUNNING, 2 TERMINATED)
+------------------------+------------+-----------------+-------------------------------+-----------------+-------------------------------+-------------+
| Trial name             | status     | loc             |   gradient_accumulation_steps |   learning_rate |   per_device_train_batch_size |   objective |
|------------------------+------------+-----------------+-------------------------------+-----------------+-------------------------------+-------------|
| _objective_c48e9_00002 | RUNNING    | 172.28.0.2:1052 |                             1 |     7.87566e-05 |                             2 |             |
| _objective_c48e

In [ ]:
!python run_language_modeling.py --output_dir=models/elonmusk --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/elonmusk/train.txt --eval_data_file=data/elonmusk/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

11/29/2021 21:57:38 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2021 21:57:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Andrej Karpathy

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/karpathy \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/karpathy/train.txt \
  --eval_data_file=data/karpathy/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

Streaming output truncated to the last 5000 lines.


(ImplicitFunc pid=1551) {'train_runtime': 159.618, 'train_samples_per_second': 50.22, 'train_steps_per_second': 12.555, 'train_loss': 3.5625168901241704, 'epoch': 3.0}
Result for _objective_3640f_00003:
  date: 2021-11-19_22-01-21
  done: true
  epoch: 3.0
  eval_loss: 3.7516210079193115
  eval_runtime: 2.4525
  eval_samples_per_second: 311.112
  eval_steps_per_second: 39.144
  experiment_id: 2418950cb1e841a08e0c33a14d9b6724
  hostname: b5c728523e37
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 3.7516210079193115
  pid: 1551
  time_since_restore: 168.7035608291626
  time_this_iter_s: 168.7035608291626
  time_total_s: 168.7035608291626
  timestamp: 1637359281
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 3640f_00003
  
(ImplicitFunc pid=1551) {'eval_loss': 3.7516210079193115, 'eval_runtime': 2.4525, 'eval_samples_per_second': 311.112, 'eval_steps_per_second': 39.144, 'epoch': 3.0}
(raylet) /usr/lo

In [ ]:
!python run_language_modeling.py --output_dir=models/karpathy --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/karpathy/train.txt --eval_data_file=data/karpathy/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

10/26/2021 02:16:07 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:16:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Barack Obama

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/BarackObama \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/BarackObama/train.txt \
  --eval_data_file=data/BarackObama/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

In [ ]:
T!python run_language_modeling.py --output_dir=models/BarackObama --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/BarackObama/train.txt --eval_data_file=data/BarackObama/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=3

10/26/2021 02:26:06 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:26:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## WSJ

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/wsj \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/wsj/train.txt \
  --eval_data_file=data/wsj/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

In [ ]:
!python run_language_modeling.py --output_dir=models/wsj --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/wsj/train.txt --eval_data_file=data/wsj/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=2 --save_strategy=epoch

10/26/2021 03:16:38 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 03:16:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## CNN

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/cnn \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/cnn/train.txt \
  --eval_data_file=data/cnn/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

In [ ]:
!python run_language_modeling.py --output_dir=models/cnn --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/cnn/train.txt --eval_data_file=data/cnn/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=2 --save_strategy=epoch

10/26/2021 03:40:09 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 03:40:09 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

# Generation and Metrics

## Elon Musk

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/elonmusk" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:28 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Am excited about the Model 3 unveil and road trip this weekend. There are some huge changes coming to the design as well, so this one is easy & fast. 
=== GENERATED SEQUENCE 2 ===
To be clear, I am against artificial intelligence. AI should not be in charge. I want it to be. 
=== GENERATED SEQUENCE 3 ===
The last remaining soul mate of the late King Arthur was actually Dragan Motavalli. He was one of the greats. 
=== GENERATED SEQUENCE 4 ===
My kids love  @SpaceX  and  @NASA. 
=== GENERATED SEQUENCE 5 ===
It's not just me: I have several cats. I also own a chicken

In [28]:
!python run_metrics.py \
  --model_type gpt2 \
  --model_name_or_path "models/elonmusk" \
  --test_data_file "data/elonmusk/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

/usr/lib/python3.7/json/encoder.py:257: UserWarning: date_default is deprecated since jupyter_client 7.0.0. Use jupyter_client.jsonutil.json_default.
  return _iterencode(o, 0)


12/01/2021 20:02:18 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='gpt2', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/elonmusk/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some

## Andrej Karpathy

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/karpathy" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:40 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
"How to Start a Startup" from  @OpenAI  great resource, tips & tricks! 
=== GENERATED SEQUENCE 2 ===
A new Deep Learning tutorial for beginners:  the DeepMind GTC231n video   (w)’s A New Challenge for Computer Visionists 
=== GENERATED SEQUENCE 3 ===
The Deep Web is a Hotbed of Violence and Corruption  soooo excited! 
=== GENERATED SEQUENCE 4 ===
#cvpr2016  a week of hacking on the ConvNetCVPR code. Looks like I'm on a roll with the demos 
=== GENERATED SEQUENCE 5 ===
It's quite amusing to watch what is happening behind closed doors with no public hearings or publ

In [29]:
!python run_metrics.py \
  --model_type gpt2 \
  --model_name_or_path "models/karpathy" \
  --test_data_file "data/karpathy/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

/usr/lib/python3.7/json/encoder.py:257: UserWarning: date_default is deprecated since jupyter_client 7.0.0. Use jupyter_client.jsonutil.json_default.
  return _iterencode(o, 0)


12/01/2021 20:04:02 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='gpt2', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/karpathy/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some

## Barack Obama

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/BarackObama" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:52 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/BarackObama', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
“They’re going to try to scare us off education entirely, because no one wants to go to college.” 
=== GENERATED SEQUENCE 2 ===
"That said, I know that people can still make their voices heard in Washington. I know it's not easy to do that." —President Obama 
=== GENERATED SEQUENCE 3 ===
The Senate must make a timely vote on an American Jobs Act now. The Jobs Act needs to be included in any deal. 
=== GENERATED SEQUENCE 4 ===
There's still more work to do, but this is progress. 
=== GENERATED SEQUENCE 5 ===
It's up to us to work with leaders in both parties to 

In [30]:
!python run_metrics.py \
  --model_type gpt2 \
  --model_name_or_path "models/BarackObama" \
  --test_data_file "data/BarackObama/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

/usr/lib/python3.7/json/encoder.py:257: UserWarning: date_default is deprecated since jupyter_client 7.0.0. Use jupyter_client.jsonutil.json_default.
  return _iterencode(o, 0)


12/01/2021 20:05:00 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/BarackObama', model_type='gpt2', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/BarackObama/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg

## CNN

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/cnn" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 04:04:59 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/cnn', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Obama: The "threat" to national security from North Korea is not unique, "from my conversations with all my colleagues, from this administration, and from the President himself" 
=== GENERATED SEQUENCE 2 ===
A new study finds that babies born with a rare form of brain cancer can develop better health than their non-nuclear counterparts 
=== GENERATED SEQUENCE 3 ===
The Senate has passed a resolution to censure President Trump for calling former President Joe Biden a 'narcissistic prick' 
=== GENERATED SEQUENCE 4 ===
 @HillaryClinton  and Donald Trump campaign to attend

In [31]:
!python run_metrics.py \
  --model_type gpt2 \
  --model_name_or_path "models/cnn" \
  --test_data_file "data/cnn/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

/usr/lib/python3.7/json/encoder.py:257: UserWarning: date_default is deprecated since jupyter_client 7.0.0. Use jupyter_client.jsonutil.json_default.
  return _iterencode(o, 0)


12/01/2021 20:05:56 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/cnn', model_type='gpt2', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/cnn/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 5-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib

## WSJ

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/wsj" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 04:05:10 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/wsj', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
‘They’re going to attack again, and it’s going to be a real bloodbath,’ says one Republican in Tennessee. ‘It’s going to be devastating.’ <
=== GENERATED SEQUENCE 2 ===
A new study finds that the number of overweight people is rising.  via  @WSJNY 
=== GENERATED SEQUENCE 3 ===
The Senate tax bill was expected to pass Thursday, but Democrats are delaying key provisions 
=== GENERATED SEQUENCE 4 ===
Opinion: Why the Supreme Court hasn’t heard the case on gay marriage. writes  @wjmcgurn 
=== GENERATED SEQUENCE 5 ===
How to be more ambitious: 


In [32]:
!python run_metrics.py \
  --model_type gpt2 \
  --model_name_or_path "models/wsj" \
  --test_data_file "data/wsj/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

/usr/lib/python3.7/json/encoder.py:257: UserWarning: date_default is deprecated since jupyter_client 7.0.0. Use jupyter_client.jsonutil.json_default.
  return _iterencode(o, 0)


12/01/2021 20:06:54 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/wsj', model_type='gpt2', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/wsj/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some weights o